In [1]:
""" 
examine why brd list contents didn't all match up to
SEC > 100M  annual reports


2019 SEC filings with 2020 bankruptcies
"""

" examine why brd list contents didn't all match up t\nSEC > 100M  annual reports"

In [36]:
import psycopg2 as pg
import pandas.io.sql as pd_sql
import pandas as pd
import numpy as np
import datetime


In [37]:
# Postgres info to connect

connection_args = {
    'host': 'localhost',  
    'dbname': 'bankruptcy',  
    'port': 5432          
}

connection = pg.connect(**connection_args)
connection.autocommit = True   
                              

In [38]:
# few fields for first initial model
# all 2019 data held out for test set

query = """
SELECT *
FROM sec_prep_2019
;
"""

sec_data = pd_sql.read_sql(query, connection)
sec_data.head()

,company_name,period_end_date,number_of_quarters,submission_number,central_index_key,ein,sic,fiscal_year_end,fiscal_year,form,...,InterestExpense,InventoryNet,SalesRevenueGoodsNet,WorkingCapital,Revenue_any,SIC Code,Industry,Unnamed: 2,div_code,Division
0,1 800 FLOWERS COM INC,2019-06-30,4,0001437749-19-018360,1084869,113117311,5990,630.0,2019,10-K,...,NaN,NaN,NaN,NaN,NaN,5990,"Retail-Retail Stores, NEC",None,59,Retail Trade
1,"10X GENOMICS, INC.",2019-12-31,4,0001193125-20-052640,1770787,455614458,3826,NaN,2019,10-K,...,2.099667e+06,11920000.0,NaN,NaN,1.544303e+08,3826,Laboratory Analytical Instruments,None,38,Manufacturing
2,"1347 PROPERTY INSURANCE HOLDINGS, INC.",2019-12-31,4,0001493152-20-005206,1591890,461119100,6331,NaN,2019,10-K,...,NaN,NaN,NaN,NaN,NaN,6331,"Fire, Marine & Casualty Insurance",None,63,"Finance, Insurance and Real Estate"
3,"1895 BANCORP OF WISCONSIN, INC.",2019-12-31,4,0001564590-20-014188,1751692,000000000,6036,NaN,2019,10-K,...,4.583000e+06,NaN,NaN,NaN,NaN,6036,"Savings Institutions, Not Federally Chartered",None,60,"Finance, Insurance and Real Estate"
4,1LIFE HEALTHCARE INC,2019-12-31,4,0001564590-20-013666,1404123,760707204,8011,NaN,2019,10-K,...,7.040000e+05,3521500.0,NaN,NaN,NaN,8011,Services-Offices & Clinics of Doctors of Medicine,None,80,Services


In [39]:
company_unique = pd.DataFrame(sec_data['company_name'].unique(), columns={'Name'})

In [40]:
company_unique = set(sec_data['company_name'].unique())
company_unique

{'IDACORP INC',
 'WAYSIDE TECHNOLOGY GROUP, INC.',
 'KBS REAL ESTATE INVESTMENT TRUST II, INC.',
 'FACTSET RESEARCH SYSTEMS INC',
 'CABOT MICROELECTRONICS CORP',
 'UNITED INSURANCE HOLDINGS CORP.',
 'HALOZYME THERAPEUTICS, INC.',
 'TRACTOR SUPPLY CO /DE/',
 'NATIONAL CINEMEDIA, INC.',
 'RESOURCE REAL ESTATE OPPORTUNITY REIT, INC.',
 'PRIMEENERGY RESOURCES CORP',
 'UNIVERSAL HEALTH REALTY INCOME TRUST',
 'SCHNEIDER NATIONAL, INC.',
 'WHITING PETROLEUM CORP',
 'FIESTA RESTAURANT GROUP, INC.',
 'MARTEN TRANSPORT LTD',
 'UNITED STATES STEEL CORP',
 'INDEPENDENT BANK CORP /MI/',
 'CIMPRESS N.V.',
 'MANNING & NAPIER, INC.',
 'MESA AIR GROUP INC',
 'LENDINGTREE, INC.',
 'VIRTU FINANCIAL, INC.',
 'STANDEX INTERNATIONAL CORP/DE/',
 'NORTHWESTERN CORP',
 'ENERGY 11, L.P.',
 'MOSAIC CO',
 'BARNES & NOBLE EDUCATION, INC.',
 'ACELRX PHARMACEUTICALS INC',
 'EDWARDS LIFESCIENCES CORP',
 'RPT REALTY',
 'YEXT, INC.',
 'TITAN MACHINERY INC.',
 'MUELLER INDUSTRIES INC',
 'DIAMOND OFFSHORE DRILLING, INC.'

In [45]:
#brd_list
brd_labels = pd.read_excel('debtor_list_ein_lookup.xlsx', sheet_name='LIST_2020_2')
brd_labels.head()

,debtor_name,date_filed,chapter_filing,name_in_sec_data,Unnamed: 4,Unnamed: 5
0,"AAC Holdings, Inc.",2020-06-20,Chapter 11,"AAC HOLDINGS, INC.",NaN,NaN
1,"Akorn, Inc.",2020-05-20,Chapter 11,AKORN INC,NaN,NaN
2,"Ascena Retail Group, Inc.",2020-07-23,Chapter 11,"ASCENA RETAIL GROUP, INC.",NaN,NaN
3,Briggs & Stratton Corporation,2020-07-20,Chapter 11,BRIGGS & STRATTON CORP,NaN,NaN
4,California Resources Corporation,2020-07-15,Chapter 11,CALIFORNIA RESOURCES CORP,NaN,NaN


In [46]:
#capitalize 
brd_labels['company_name_brd'] = brd_labels['name_in_sec_data'].str.upper()
brd_labels.head()

,debtor_name,date_filed,chapter_filing,name_in_sec_data,Unnamed: 4,Unnamed: 5,company_name_brd
0,"AAC Holdings, Inc.",2020-06-20,Chapter 11,"AAC HOLDINGS, INC.",NaN,NaN,"AAC HOLDINGS, INC."
1,"Akorn, Inc.",2020-05-20,Chapter 11,AKORN INC,NaN,NaN,AKORN INC
2,"Ascena Retail Group, Inc.",2020-07-23,Chapter 11,"ASCENA RETAIL GROUP, INC.",NaN,NaN,"ASCENA RETAIL GROUP, INC."
3,Briggs & Stratton Corporation,2020-07-20,Chapter 11,BRIGGS & STRATTON CORP,NaN,NaN,BRIGGS & STRATTON CORP
4,California Resources Corporation,2020-07-15,Chapter 11,CALIFORNIA RESOURCES CORP,NaN,NaN,CALIFORNIA RESOURCES CORP


In [47]:
brd_names = set(brd_labels['company_name_brd'])
brd_names

{'AAC HOLDINGS, INC.',
 'AKORN INC',
 'ASCENA RETAIL GROUP, INC.',
 'BRIGGS & STRATTON CORP',
 'CALIFORNIA RESOURCES CORP',
 'CARBO CERAMICS INC',
 'CBL & ASSOCIATES PROPERTIES INC',
 'CEC ENTERTAINMENT INC',
 'CENTRIC BRANDS INC.',
 'CHAPARRAL ENERGY, INC.',
 'CHESAPEAKE ENERGY CORP',
 'COVIA HOLDINGS CORP',
 'DENBURY RESOURCES INC',
 'DIAMOND OFFSHORE DRILLING, INC.',
 'EXTRACTION OIL & GAS, INC.',
 'FORESIGHT ENERGY LP',
 'FRONTIER COMMUNICATIONS CORP',
 'FTS INTERNATIONAL, INC.',
 'GLOBAL EAGLE ENTERTAINMENT INC.',
 'GNC HOLDINGS, INC.',
 'GULFPORT ENERGY CORP',
 'HERTZ CORP',
 'HERTZ GLOBAL HOLDINGS INC',
 'HERTZ GLOBAL HOLDINGS, INC',
 'HI-CRUSH INC.',
 'HORNBECK OFFSHORE SERVICES INC /LA',
 'INTERNAP CORP',
 'J C PENNEY CO INC',
 'J CREW GROUP INC',
 'JASON INDUSTRIES, INC.',
 'LIBBEY INC',
 'LONESTAR RESOURCES US INC.',
 'LSC COMMUNICATIONS, INC.',
 'MCCLATCHY CO',
 'MCDERMOTT INTERNATIONAL INC',
 'NEIMAN MARCUS GROUP LTD LLC',
 'OASIS PETROLEUM INC.',
 'PENNSYLVANIA REAL ESTAT

In [48]:
# compare the two sets 

brd_names.difference(company_unique)

{'AAC HOLDINGS, INC.',
 'CENTRIC BRANDS INC.',
 'HERTZ CORP',
 'HERTZ GLOBAL HOLDINGS INC',
 'NEIMAN MARCUS GROUP LTD LLC'}

In [ ]:
### Have notes in word doc.  these are okay

In [35]:
query = """
SELECT distinct company_name, period_end_date
FROM sec_prep_2019
WHERE  company_name LIKE '%DIAMOND%'
;
"""

one_company = pd_sql.read_sql(query, connection)
one_company

,company_name,period_end_date
0,"BLACK DIAMOND THERAPEUTICS, INC.",2019-12-31
1,DIAMOND EAGLE ACQUISITION CORP. DE,2019-12-31
2,DIAMOND HILL INVESTMENT GROUP INC,2019-12-31
3,"DIAMOND OFFSHORE DRILLING, INC.",2019-12-31
4,DIAMOND S SHIPPING INC.,2019-12-31
5,"DIAMONDBACK ENERGY, INC.",2019-12-31
6,DIAMONDPEAK HOLDINGS CORP.,2019-12-31
7,DIAMONDROCK HOSPITALITY CO,2019-12-31


In [31]:
query = """
SELECT distinct company_name
FROM sec_all_raw
WHERE  company_name LIKE '%HI-%'
;
"""

one_company = pd_sql.read_sql(query, connection)
one_company


,company_name
0,HI-CRUSH PARTNERS LP


In [11]:
query = """
SELECT distinct company_name, period_end_date
FROM sec_all_raw
WHERE  company_name LIKE '%ASCENA RETAIL GROUP, INC.%'
;
"""

one_company = pd_sql.read_sql(query, connection)
one_company


,company_name,period_end_date
0,"ASCENA RETAIL GROUP, INC.",2011-07-31
1,"ASCENA RETAIL GROUP, INC.",2012-07-31
2,"ASCENA RETAIL GROUP, INC.",2013-07-31
3,"ASCENA RETAIL GROUP, INC.",2014-07-31
4,"ASCENA RETAIL GROUP, INC.",2015-07-31
5,"ASCENA RETAIL GROUP, INC.",2016-07-31
6,"ASCENA RETAIL GROUP, INC.",2017-07-31
7,"ASCENA RETAIL GROUP, INC.",2018-07-31


In [12]:

query = """
SELECT distinct company_name, form, number_of_quarters, period_end_date
FROM sec_raw_all
WHERE  company_name LIKE '%CENTRIC BRANDS INC.'
;
"""

one_company = pd_sql.read_sql(query, connection)
one_company



,company_name,form,number_of_quarters,period_end_date
0,CENTRIC BRANDS INC.,10-K,0,2016-12-31
1,CENTRIC BRANDS INC.,10-K,0,2017-12-31
2,CENTRIC BRANDS INC.,10-K,0,2018-12-31
3,CENTRIC BRANDS INC.,10-K,4,2017-12-31
4,CENTRIC BRANDS INC.,10-K,4,2018-12-31


In [57]:

query = """
SELECT distinct company_name, form, number_of_quarters, period_end_date
FROM sec_raw_all
WHERE  period_end_date < '2019-01-01' 
    and company_name LIKE '%SUNEDISON, INC%'
;
"""

one_company = pd_sql.read_sql(query, connection)
one_company

# bankruptcy 4/21/2016
# have 10-Q 	2015-09-30

,company_name,form,number_of_quarters,period_end_date
0,"SUNEDISON, INC.",10-K,0,2010-12-31
1,"SUNEDISON, INC.",10-K,0,2011-12-31
2,"SUNEDISON, INC.",10-K,0,2012-12-31
3,"SUNEDISON, INC.",10-K,0,2013-12-31
4,"SUNEDISON, INC.",10-K,0,2014-12-31
5,"SUNEDISON, INC.",10-K,4,2011-12-31
6,"SUNEDISON, INC.",10-K,4,2012-12-31
7,"SUNEDISON, INC.",10-K,4,2013-12-31
8,"SUNEDISON, INC.",10-K,4,2014-12-31


In [76]:

query = """
SELECT distinct company_name, form, number_of_quarters, period_end_date
FROM sec_raw_all
WHERE  period_end_date < '2019-01-01' 
    and company_name LIKE '%VENOCO%'
;
"""

one_company = pd_sql.read_sql(query, connection)
one_company
#bankruptcy 3/18/2016  - some filings are VENOCO other filings are DENVER PARENT CORP
#BRD list has Venoco, Inc. (Denver Parent Corp.)
    

,company_name,form,number_of_quarters,period_end_date
0,"VENOCO, INC.",10-K,0,2010-12-31
1,"VENOCO, INC.",10-K,0,2011-12-31
2,"VENOCO, INC.",10-K,0,2012-12-31
3,"VENOCO, INC.",10-K,0,2013-12-31
4,"VENOCO, INC.",10-K,1,2012-03-31
5,"VENOCO, INC.",10-K,1,2012-06-30
6,"VENOCO, INC.",10-K,1,2012-09-30
7,"VENOCO, INC.",10-K,1,2012-12-31
8,"VENOCO, INC.",10-K,1,2013-03-31
9,"VENOCO, INC.",10-K,1,2013-06-30


In [75]:

query = """
SELECT distinct company_name, form, number_of_quarters, period_end_date
FROM sec_raw_all
WHERE  period_end_date < '2019-01-01' 
    and company_name LIKE '%DENVER PARENT%'
;
"""

one_company = pd_sql.read_sql(query, connection)
one_company
 #bankruptcy 3/18/2016  - some filings are VENOCO

,company_name,form,number_of_quarters,period_end_date
0,DENVER PARENT CORP,10-K,0,2011-12-31
1,DENVER PARENT CORP,10-K,0,2012-12-31
2,DENVER PARENT CORP,10-K,0,2013-12-31
3,DENVER PARENT CORP,10-K,0,2014-12-31
4,DENVER PARENT CORP,10-K,0,2015-12-31
5,DENVER PARENT CORP,10-K,1,2013-03-31
6,DENVER PARENT CORP,10-K,1,2013-06-30
7,DENVER PARENT CORP,10-K,1,2013-09-30
8,DENVER PARENT CORP,10-K,1,2013-12-31
9,DENVER PARENT CORP,10-K,1,2014-03-31
